### 第三章 MDP：Finite Markov Decision Process（有限马尔可夫决策过程）

所谓马尔可复决策过程（__MDP__）是指一个五元组$(S, A, P, R, \gamma)$，其中：

- $S$是状态集合
- $A$是动作集合
- $P$是状态转移概率矩阵
- $R$是奖励函数
- $\gamma$是折扣因子

_（有限马尔可夫中的 __“有限”__ 表示上述状态集合和动作集合都是有限的，而非无限情况下的。）_

MDP中，动作不仅影响当前的即时收益，还影响后续的情境（又称状态）以及未来的收益。因此, MDP 涉及了 __延迟收益__ ，由此也就有了在当前收益和延迟收益之间权衡的需求。在上一章赌博机问题中，我们估计了每个动作 ${a}$ 的价值 $Q(a)$; 而在 MDP 中，我们估计了：
- 每个动作 ${a}$ 在每个状态 ${s}$ 中的价值 $Q(s, a)$, 或者
- 估计给定最优动作下的每个状态的价值 $V(s)$。 

并且这些价值都是长期累积的。

MDP是一个非常重要的强化学习模型，它描述了一个智能体与环境交互的过程。智能体在某个状态$s$下执行动作$a$，环境根据状态转移概率$P$将智能体的状态从$s$转移到$s'$，并给予智能体奖励$R$。智能体的目标是最大化长期累积奖励，即最大化折扣累积奖励$\sum_{t=0}^{\infty} \gamma^t r_t$。



#### 3.1 The Agent-environment Interface（智能体-环境接口）

MDP 就是一种通过交互式学习来实现目标的理论框架。那么引出：

- 什么是智能体 agent
- 什么是环境 environment

智能体选择动作，环境对这些动作做出相应的响应，并向智能体呈现出新的状态。环境也会产生一个收益，通常是特定的数值，表示智能体在这个状态下的表现。智能体的目标是最大化这个收益。如下图所示：

![MDP](img_1.png)

__MDP的公式化定义如下：__

$$
p(s',r \mid s,a) \doteq \Pr\{S_t = s', R_t = r \mid S_{t-i} = s, A_{t-i} = a\}
$$

这里，$p(s',r \mid s,a)$是 __状态转移概率__ 动态函数，表示在状态$s$下执行动作$a$后，智能体将会转移到状态$s'$并获得奖励$r$的概率。


（动态函数 $p: S \times A \times R \times S \rightarrow [0, 1]$ 来自行为的序列或轨迹：$S_0, A_0, R_1, S_1, A_1, R_2, S_2, A_2, R_3, \ldots$ ）


函数 p 为每个 s 和 a 的选择都指定了一个概率分布，即：

$$
\sum_{s' \in S} \sum_{r \in R} p(s',r \mid s,a) = 1, \quad \text{对于所有 } s \in S, a \in A(s)
$$


也就是说，$S_t$ 和 $R_t$ 的每个可能的值出现的概率只取决于前一个状态 $S_{t一1}$ 和前一个动作 $A_{t一1}$, 并且与更早之前的状态和动作完全无关。这个限制是针对状态的。状态必须包括过去智能体和环境交互的方方面面的信息，这些信息会对未来产生一定影响。这样，状态就被认为具有 __马尔可夫性__ 。

__MDP框架的限制：__

_非静态环境_：MDP假设环境和奖励函数是静态的，即它们不会随着时间变化。如果环境是非静态的，MDP框架可能无法充分表示任务。在这些情况下，自适应或在线学习技术可能更合适。

__例子：Recycling Robot 扫地机器人：__

状态、动作、下一状态、下一状态转移概率、奖励函数等信息整理成表格，如下图所示：

（左侧）表格归纳图、（右侧）转移归纳图

![Recycling Robot](img.png)

#### 3.2 Goals and Rewards（目标和奖励）

- 在强化学习中，智能体的目标是最大化长期累积奖励（即最大化其收到的总收益）。
- 如何设计奖励值？胜利+1、失败-1？胜利+100，每一步-1？等等。
- 设计最终目标的重要性。赢棋？通关？击败对手？等等。

#### 3.3 Returns and Episodes（回报和回合）

回报是收益的综合：

$$
G_t \doteq R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \gamma^3 R_{t+4} + \ldots = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}
$$

其中，$G_t$是从时间步 $t$ 开始的折扣累积奖励，$\gamma$是一个 $0 \leq \gamma \leq 1$ 的折扣因子。折扣因子决定了未来奖励的重要性。$\gamma$ 越接近 0，智能体越关注即时奖励；$\gamma$ 越接近 1，智能体越关注长期奖励。公式通过递归方式，可以简化为：

$$
\begin{align*}
G_t &\doteq R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \gamma^3 R_{t+4} + \cdots \\
&= R_{t+1} + \gamma ( R_{t+2} + \gamma R_{t+3} + \gamma^2 R_{t+4} + \cdots ) \\
&= R_{t+1} + \gamma G_{t+1}
\end{align*}
$$



书中举例，假如收益 $R$ 是一个常数 $+1$ ，那么通过几何级数的求和公式可以得出：

$$
G_t = \sum_{k=0}^{\infty} \gamma^k = \frac{1}{1 - \gamma}.
$$

它想说明的是，即使回报是对无限个收益子项求和，但只要收益是一个非零常数且 $\gamma <1$ ,那这个回报仍是有限的。

而 __episode “回合”__ 的概念更加直接，就是在环境当中的每个任务的回合。比如agent赢下一局五子棋游戏，就是一个“回合”；agent走出一次迷宫，就是一次“回合”。


#### __例子：Pole-Balancing 平衡车游戏：__

一个典型的多回合（episode）强化学习例子。在这个例子中，智能体的目标是保持杆子竖直。每个回合开始时，杆子是竖直的，智能体必须选择动作来保持杆子竖直。如果杆子倾斜超过一定角度，智能体就会失败。智能体的目标是尽可能多地保持杆子竖直，直到达到某个时间步数或者直到智能体失败。在这个例子中，每个回合都是一个任务，智能体的目标是最大化每个回合的回报。

In [22]:
# 使用OpenAI的gym库创建一个游戏环境
import gymnasium as gym
import numpy as np

# 创建 CartPole 环境
env = gym.make('CartPole-v1', render_mode="human")

# 设置回合数
n_episodes = 20

for episode in range(n_episodes):
    observation, info = env.reset()  # 重置环境，获取状态、信息
    total_reward = 0
    done = False
    truncated = False

    while not (done or truncated):
        # 渲染环境
        env.render()

        # 随机选择一个动作（0：向左移动，1：向右移动）
        action = env.action_space.sample()

        # 执行动作
        observation, reward, done, truncated, info = env.step(action)
        
        # 累计奖励
        total_reward += reward

    print(f"Episode {episode + 1}: Total Reward = {total_reward}")

# 关闭环境
env.close()

2024-09-29 21:43:45.197 python[5669:334002] +[IMKClient subclass]: chose IMKClient_Legacy
2024-09-29 21:43:45.197 python[5669:334002] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Episode 1: Total Reward = 31.0
Episode 2: Total Reward = 18.0
Episode 3: Total Reward = 17.0
Episode 4: Total Reward = 15.0
Episode 5: Total Reward = 17.0
Episode 6: Total Reward = 15.0
Episode 7: Total Reward = 35.0
Episode 8: Total Reward = 12.0
Episode 9: Total Reward = 10.0
Episode 10: Total Reward = 11.0
Episode 11: Total Reward = 11.0
Episode 12: Total Reward = 46.0
Episode 13: Total Reward = 13.0
Episode 14: Total Reward = 10.0
Episode 15: Total Reward = 15.0
Episode 16: Total Reward = 24.0
Episode 17: Total Reward = 8.0
Episode 18: Total Reward = 19.0
Episode 19: Total Reward = 23.0
Episode 20: Total Reward = 23.0


#### 3.4 Unified Notation for Episodic and Continuing Tasks（回合任务和连续任务的统一符号表示）

这一章中主要讨论的是强化学习中智能体与环境交互的过程。在这个过程中，智能体与环境交互的任务可以分为两种类型：__回合任务__ 和 __连续任务__。

- 回合任务：比如一局游戏
- 连续任务：比如一只机械臂连续作业。。

这章主要讲统一符号，及在表达式中忽略Episode的标记 ${i}$。

书中为了简略，将 $R_{t,i}$ 表示为 $R_{t}$，将 $S_{t,i}$ 表示为 $S_{t}$ ，将 $A_{t,i}$ 表示为 $A_{t}$ 等等

#### 3.5 Policies and Value Functions（策略和价值函数）


概念： __价值函数__：Agent在给定状态（或状态-动作对）下 “能有多好” 的函数。而 __策略__：Agent在给定状态下 “应该做什么” 的函数。 策是从状态到每个动作的选择概率之间的映射。如果Agent在 ${t}$ 时刻选择了策略 $\pi$ ，那么 $\pi(a|s)$ 就是在状态 ${s}$ 下选择动作 ${a}$ 的概率。

以下为两种价值函数的定义：

- 策略价值函数 $v_{\pi}(s)$ 是在策略 $\pi$ 下状态 ${s}$ 的价值：
$$
v_{\pi}(s) \doteq \mathbb{E}_{\pi} [G_t \mid S_t = s] = \mathbb{E}_{\pi} \left[ \sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \mid S_t = s \right]
$$
- 动作价值函数 $q_{\pi}(s, a)$ 是在策略 $\pi$ 下状态 ${s}$ 和动作 ${a}$ 的价值：
$$
q_{\pi}(s, a) \doteq \mathbb{E}_{\pi} [G_t \mid S_t = s, A_t = a] = \mathbb{E}_{\pi} \left[ \sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \mid S_t = s, A_t = a \right]
$$


#### Bellman Expectation Equation（贝尔曼期望方程）

- 策略价值函数 $v_{\pi}(s)$ 是在策略 $\pi$ 下状态 ${s}$ 的价值：

$$
v_{\pi}(s) = \sum_{a} \pi(a|s) \sum_{s', r} p(s', r \mid s, a) [r + \gamma v_{\pi}(s')]
$$

- 动作价值函数 $q_{\pi}(s, a)$ 是在策略 $\pi$ 下状态 ${s}$ 和动作 ${a}$ 的价值：

$$
q_{\pi}(s, a) = \sum_{s', r} p(s', r \mid s, a) [r + \gamma \sum_{a'} \pi(a'|s') q_{\pi}(s', a')]
$$

简而言之，整个方程的含义是：

在某个状态下，按照策略π选择动作，然后考虑所有可能的下一个状态和奖励，__计算即时奖励加上折扣的未来价值的期望总和__。


#### __例子：Gridworld 网格游戏：__

实现书中 example 3.5 的网格游戏，如下图所示：

![image](img_2.png)

In [21]:
import numpy as np
import plotly.graph_objects as go

# 根据状态和动作获取奖励
def get_reward(state: tuple, act: tuple):
    """
    state: 当前状态，元组 (行, 列)
    act: 动作，元组 (行变化, 列变化)
    
    返回: 奖励和下一个状态
    """
    if state == (0, 1):
        return 10, (4, 1)
    if state == (0, 3):
        return 5, (2, 3)

    next_row = state[0] + act[0]
    next_col = state[1] + act[1]

    if not (0 <= next_row < 5 and 0 <= next_col < 5):
        return -1, state

    return 0, (next_row, next_col)

# 价值函数更新
def grid_state_value_func(grid_world: np.ndarray, actions: list, gamma: float):
    """
    grid_world: 5x5网格世界地图，numpy数组
    actions: 可能的动作列表
    gamma: 折扣因子 0.9
    
    :returns
    格子及其累积的价值。
    【说明】：每个格子的值是基于当前状态下所有可能动作的期望值计算得到的。这个期望值是通过对每个可能动作的奖励和下一个状态的折扣值进行加权平均计算得出。
    """
    new_grid = np.copy(grid_world)
    for row in range(5):
        for col in range(5):
            curr_value = 0
            for act in actions:
                reward, next_state = get_reward((row, col), act)
                next_value = grid_world[next_state]
                curr_value += 0.25 * (reward + gamma * next_value) # R + gamme * R_{t+1}
            new_grid[row, col] = curr_value
    return new_grid

# 绘制价值表
def plot_grid(grid_world: np.ndarray, iteration: int = 0):
    """
    grid_world: The grid world to be plotted
    iteration: Current iteration count
    """
    # Create the heatmap with improved colorscale and font settings
    fig = go.Figure(data=go.Heatmap(
        z=grid_world,
        text=[[f'{val:.1f}' for val in row] for row in grid_world],
        texttemplate="%{text}",
        textfont={"size": 16, "color": "white"},
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='Value', titleside='right', ticks='outside')
    ))

    # Update layout for aesthetics and better formatting
    fig.update_layout(
        title=f'Iteration {iteration}',
        title_font_size=12,
        title_x=0.5,
        width=450,
        height=450,
        margin=dict(l=50, r=50, t=100, b=50),  # Add margins for better visualization
        paper_bgcolor='rgba(0,0,0,0)',  # Transparent background
        plot_bgcolor='rgba(0,0,0,0)',   # Transparent plot area
        xaxis=dict(showticklabels=False),
        yaxis=dict(showticklabels=False, autorange='reversed')  # Reverse y-axis
    )

    # Adding gridlines for a cleaner look
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

    fig.show()

# 初始化网格世界
grid_world = np.zeros(shape=(5, 5))
actions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # 右、左、下、上 4种动作
gamma = 0.9  # 折扣因子

# 主循环
"""
通过 np.allclose() 函数来检查两个数组是否非常接近，如果是，则停止迭代。
当网格近似相等时，迭代停止，因为这表明数值已经收敛。
在马尔可夫决策过程（MDP）的价值迭代中，收敛意味着进一步的迭代不会显著改变网格中的数值。
表明算法已经找到一个稳定的解，继续迭代将变得不必要且浪费计算资源。
"""
for i in range(40):
    prev_grid = np.copy(grid_world)
    grid_world = grid_state_value_func(grid_world, actions, gamma)

    if np.allclose(prev_grid, grid_world, rtol=0.01):
        # 匹配书中的结果：
        print(f'完成于第 {i} 次迭代')
        plot_grid(grid_world, i)
        break

完成于第 30 次迭代


#### __例子：Golf 高尔夫球游戏：__

实现书中 example 3.6 的高尔夫游戏，如下图所示：

![image](img_3.png)

In [20]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 定义高尔夫球场环境
class GolfEnv:
    def __init__(self):
        self.green_area = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 0), (0, 1)]
        self.hole = (0, 0)
        self.max_distance = 6

    def step(self, state, action):
        x, y = state
        dx, dy = action
        new_x, new_y = x + dx, y + dy

        # 检查是否进洞
        if (new_x, new_y) == self.hole:
            return (new_x, new_y), 0, True

        # 检查是否在绿色区域
        if (new_x, new_y) in self.green_area:
            return (new_x, new_y), -1, False

        # 检查是否出界
        if abs(new_x) > self.max_distance or abs(new_y) > self.max_distance:
            return state, -1, False

        return (new_x, new_y), -1, False

# Q-learning 算法
def q_learning(env, episodes=10000, alpha=0.1, gamma=0.9, epsilon=0.1):
    q_table = {}

    actions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]

    for episode in range(episodes):
        state = (-6, 0)  # 起始位置
        done = False

        while not done:
            if state not in q_table:
                q_table[state] = np.zeros(len(actions))

            if np.random.random() < epsilon:
                action = actions[np.random.choice(len(actions))]
            else:
                action = actions[np.argmax(q_table[state])]

            next_state, reward, done = env.step(state, action)

            if next_state not in q_table:
                q_table[next_state] = np.zeros(len(actions))

            # Q-learning更新
            old_q = q_table[state][actions.index(action)]
            next_max = np.max(q_table[next_state])
            new_q = (1 - alpha) * old_q + alpha * (reward + gamma * next_max)
            q_table[state][actions.index(action)] = new_q

            state = next_state
            
        epsilon = max(0.01, epsilon * 0.99)  # epsilon衰减策略以便更好地平衡探索与利用
    return q_table

# 可视化Q-table和原始地图
def visualize_q_table_and_map(q_table, env):
    x = np.arange(-6, 7)
    y = np.arange(-6, 7)
    X, Y = np.meshgrid(x, y)

    Z = np.zeros_like(X, dtype=float)

    for i, xi in enumerate(x):
        for j, yi in enumerate(y):
            if (xi, yi) in q_table:
                # Z[j, i] = np.max(q_table[(xi, yi)])
                Z[j, i] = np.max(q_table.get((xi, yi), [-10]))  # 如果状态没有在 q_table 中，给一个较低值
            else:
                Z[j, i] = -10  # 为不可达状态设置一个较低的值

    fig = make_subplots(rows=1, cols=2, subplot_titles=("Q-learning热力图", "原始游戏地图"))

    # 热力图
    heatmap = go.Heatmap(z=Z, x=x, y=y, colorscale='Viridis', zmin=-10, zmax=0)
    fig.add_trace(heatmap, row=1, col=1)

    # 原始地图
    # 绿色区域
    green_x, green_y = zip(*env.green_area)
    fig.add_trace(go.Scatter(x=green_x, y=green_y, mode='markers', marker=dict(size=10, color='green'), name='绿色区域'), row=1, col=2)

    # 球洞
    fig.add_trace(go.Scatter(x=[env.hole[0]], y=[env.hole[1]], mode='markers', marker=dict(size=15, color='black', symbol='circle-open'), name='球洞'), row=1, col=2)

    # 起始位置
    fig.add_trace(go.Scatter(x=[-6], y=[0], mode='markers', marker=dict(size=10, color='blue'), name='起始位置'), row=1, col=2)

    # 等高线
    contours = go.Contour(z=Z, x=x, y=y, contours=dict(start=-10, end=0, size=1), colorscale='Viridis', showscale=False)
    fig.add_trace(contours, row=1, col=2)

    fig.update_layout(height=600, width=1200, title_text="高尔夫Q-learning状态价值函数与原始地图对比")
    fig.update_xaxes(range=[-6.5, 6.5], title_text="X坐标")
    fig.update_yaxes(range=[-6.5, 6.5], title_text="Y坐标")

    fig.show()

# 主程序
env = GolfEnv()
q_table = q_learning(env)
visualize_q_table_and_map(q_table, env)

#### 3.6 Optimal Policies and Optimal Value Functions（最优策略和最优价值函数）

#### 3.7 Optimality and Approximation（最优性和近似）

#### 3.8 Summary（小结）